In [37]:
type Action =
| WalkTo of string
| Open of string

type Graph = { 
    nodes: Map<string, Action seq>
    flow: Map<string,int>
}
module Graph =
    module Parser =
        open System.Text.RegularExpressions

        let regex = Regex(@"^Valve (?<valve>\w.) has flow rate=(?<flow>\d+); tunnels? leads? to valves? (?<valves>(?:\w.)(:?,\s\w.)*)$")

        let folder graph line : Graph =
            let m = regex.Match(line)

            let label = m.Groups.["valve"].Value
            let flow = m.Groups.["flow"].Value |> int
            let valves = 
                m.Groups.["valves"].Value.Split(", ")
                |> Seq.map Action.WalkTo
                |> Seq.append [ Action.Open label ]

            { graph with 
                nodes = graph.nodes |> Map.add label valves
                flow = graph.flow |> Map.add label flow
            }

        let parse lines =
            lines |> Seq.fold folder { nodes = Map.empty; flow = Map.empty }

    let flow graph action =
        let label = match action with WalkTo w -> w | Open o -> o
        graph.flow |> Map.find label

    let neighbours graph action =
        let label = match action with WalkTo w -> w | Open o -> o
        graph.nodes 
        |> Map.find label
    

In [7]:
let ResolutionFolder = __SOURCE_DIRECTORY__
let graph =
    File.ReadLines( ResolutionFolder + "/testcase16.txt")
    |> Graph.Parser.parse

In [38]:
type Simulation = {
    graph: Graph
    current: Action
    released: int
    steps: int
    closed: Action seq
    opened: Action seq
} with 
    member this.FlowRate () =
        this.opened
        |> Seq.map (Graph.flow this.graph)
        |> Seq.sum

module Simulation =
    let rec step acc state =